In [31]:
import numpy as np
import torch
from torch.utils.data import Dataset

rain_float_map = {
    "rain": 1.0,
    b'rain': 1.0,
    "no rain": 0.0,
    b'no rain': 0.0
}

class WeatherDataset(Dataset):
    def __init__(self, file_path="weather_forecast_data.csv", transform=None, target_transform=None):
        self.records = np.genfromtxt(
            file_path, 
            dtype="f8",
            delimiter=",", 
            skip_header=1, 
            converters={5: lambda s: rain_float_map.get(s) }
        ).astype(np.float32)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.records)

    def __getitem__(self, idx):
        record = self.records[idx]
        x = torch.from_numpy(record[:-1])
        y = torch.from_numpy(np.atleast_1d(record[-1]))
        if self.transform:
            x = self.transform(x)
        if self.target_transform:
            y = self.target_transform(y)
        return x, y



In [32]:
from torch.utils.data import DataLoader
from torch import nn
from torchvision import datasets, transforms

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

full_dataset = WeatherDataset()
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [0.8, 0.2])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

Using cpu device


In [33]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            nn.Linear(10, 1),
        )

    def forward(self, x):
        return self.linear_relu_stack(x)

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=1, bias=True)
  )
)


In [34]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Initialize the loss function
loss_fn = nn.BCEWithLogitsLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [35]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            binary_pred = torch.where(pred.sigmoid() > 0.8, 1.0, 0.0)
            correct += (binary_pred == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [36]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 24.371988  [   64/ 2000]
loss: 0.265936  [  704/ 2000]
loss: 0.386008  [ 1344/ 2000]
loss: 0.307469  [ 1984/ 2000]
Test Error: 
 Accuracy: 80.8%, Avg loss: 0.893792 

Epoch 2
-------------------------------
loss: 0.857935  [   64/ 2000]
loss: 0.324856  [  704/ 2000]
loss: 0.401703  [ 1344/ 2000]
loss: 0.157120  [ 1984/ 2000]
Test Error: 
 Accuracy: 53.2%, Avg loss: 1.433683 

Epoch 3
-------------------------------
loss: 1.425898  [   64/ 2000]
loss: 0.404796  [  704/ 2000]
loss: 0.285925  [ 1344/ 2000]
loss: 0.235496  [ 1984/ 2000]
Test Error: 
 Accuracy: 86.8%, Avg loss: 0.444768 

Epoch 4
-------------------------------
loss: 0.415818  [   64/ 2000]
loss: 0.280104  [  704/ 2000]
loss: 0.240801  [ 1344/ 2000]
loss: 0.347660  [ 1984/ 2000]
Test Error: 
 Accuracy: 86.8%, Avg loss: 0.377331 

Epoch 5
-------------------------------
loss: 0.223243  [   64/ 2000]
loss: 0.371096  [  704/ 2000]
loss: 0.295505  [ 1344/ 2000]
loss: 0.293904  [ 198